# Playing with Sqlite3

### Connecting to a Database

In [7]:
#import the sqlite module
import sqlite3 as sql

#start with a connection object
conn = sql.connect('example.db')

# create a cursor object and use its execute method to run SQL commands
c = conn.cursor()

### Create a New Table in Database

In [2]:
#execute a sql command creating a table
c.execute('''CREATE TABLE chicken (name text, dob text, salary real, favcolor text)
''')

### Insert values into a table, commit, and close

In [3]:
#insert into table
c.execute(""" INSERT INTO names VALUES ('harrison', '4/25/1995', 100000, 'blue')""")

# Commit the changes
conn.commit()

#close the connection
conn.close()

In [4]:
#reopen the connection to see if changes are still there
conn2 = sql.connect("example.db")
c = conn2.cursor()

### Using parameter substitution to specify multiple values.


In [3]:
#use a python tuple object and insert into a SQL query
names = ("harrison",)
c.execute("SELECT * FROM names WHERE name=?", names)
#fetchall shows all, fetchone showsone
c.fetchall()

[('harrison', '4/25/1995', 100000.0, 'blue'),
 ('harrison', '4/25/1995', 100000.0, 'blue'),
 ('harrison', '4/25/1995', 100000.0, 'blue'),
 ('harrison', '4/25/1995', 100000.0, 'blue')]

### Inserting with Parameter Substitutions

Without this we would have to physically write the data in as text in the SQL query being passed to the cursor object.

In [8]:
#inserting many examples with parameter substitution
inserts = [('huckle', '11/11/2016', 0, 'red'),
('chicken', '629836', 182098, 'green')]

#insert into the db with the cursor object
c.executemany("INSERT INTO names VALUES (?,?,?,?)", inserts)

#finalizes the changes into the db
conn2.commit()

('huckle', '11/11/2016', 0.0, 'red')


### Looping over rows in a table

In [12]:
#loop over rows in a database
for row in c.execute("SELECT * FROM names ORDER BY salary"):
    print(row[0]+" has bad breath.")

huckle has bad breath.
harrison has bad breath.
harrison has bad breath.
harrison has bad breath.
harrison has bad breath.
chicken has bad breath.


In [20]:
namequery = ('harrison',)
for row in c.execute("select salary from names where name==?", namequery):
    print(row[0])
    print("Salary is: $", row[0]**2, '\n')

100000.0
Salary is: $ 10000000000.0 

100000.0
Salary is: $ 10000000000.0 

100000.0
Salary is: $ 10000000000.0 

100000.0
Salary is: $ 10000000000.0 



### Printing an Entire Table

In [8]:
#show all rows in a db
c.execute("select * from names")
c.fetchall()

[('harrison', '4/25/1995', 100000.0, 'blue'),
 ('harrison', '4/25/1995', 100000.0, 'blue'),
 ('harrison', '4/25/1995', 100000.0, 'blue'),
 ('harrison', '4/25/1995', 100000.0, 'blue'),
 ('huckle', '11/11/2016', 0.0, 'red'),
 ('chicken', '629836', 182098.0, 'green')]

In [10]:
conn2.close()

# SQL and Pandas

Data is not synonymous with relational database. (Tables that look like excel files). We can have CSV files, plain text,  Parquet, and more.

Pandas is particularly geared towards structured data

### The anatomy of a SQL query

SELECT ... FROM ... WHERE

GROUP BY ... HAVING

ORDER BY ...

### Examples of Queries

In [11]:
conn = sql.connect("example.db")
c = conn.cursor()

In [12]:
#using an or opperator to select multiple conditions
c.execute("SELECT * FROM names WHERE name='harrison' OR favcolor='green'")
c.fetchall()

[('harrison', '4/25/1995', 100000.0, 'blue'),
 ('harrison', '4/25/1995', 100000.0, 'blue'),
 ('harrison', '4/25/1995', 100000.0, 'blue'),
 ('harrison', '4/25/1995', 100000.0, 'blue'),
 ('chicken', '629836', 182098.0, 'green')]

In [14]:
# using the WHERE IN combination to select multiple possible values
c.execute("SELECT * FROM names WHERE favcolor IN ('green', 'red')")
c.fetchall()

[('huckle', '11/11/2016', 0.0, 'red'),
 ('chicken', '629836', 182098.0, 'green')]

In [15]:
# group table by a column
c.execute("SELECT * FROM names GROUP BY name")
c.fetchall()

[('chicken', '629836', 182098.0, 'green'),
 ('harrison', '4/25/1995', 100000.0, 'blue'),
 ('huckle', '11/11/2016', 0.0, 'red')]

In [21]:
#group by column order by numeric column
c.execute("SELECT * FROM names GROUP BY name ORDER BY salary DESC")
c.fetchall()

[('chicken', '629836', 182098.0, 'green'),
 ('harrison', '4/25/1995', 100000.0, 'blue'),
 ('huckle', '11/11/2016', 0.0, 'red')]

### Aggregate Functions

In [21]:
#uses agg function COUNT
c.execute("SELECT *, COUNT(*) FROM names")
c.fetchall()

[('chicken', '629836', 182098.0, 'green', 6)]

In [4]:
# max function
c.execute("SELECT MAX(salary) FROM names")
c.fetchall()

[(182098.0,)]

In [5]:
conn.close()

### SQL Into Pandas

In [31]:
import pandas as pd

# first connect to the database
conn = sql.connect('example.db')

#read the sql command with the connection object into the db
df = pd.read_sql('select * from names', conn)

#display the database we imported into pandas
df

,name,dob,salary,favcolor
0,harrison,4/25/1995,100000.0,blue
1,harrison,4/25/1995,100000.0,blue
2,harrison,4/25/1995,100000.0,blue
3,harrison,4/25/1995,100000.0,blue
4,huckle,11/11/2016,0.0,red
5,chicken,629836,182098.0,green
